<a href="https://colab.research.google.com/github/kaminglui/Domain-Adaptation-with-ME-IIS/blob/main/notebooks/Run_All_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Run All Experiments (Unified UDA Runner)

Runs a clean baseline suite with a single shared runner:
- `source_only`, `dann`, `dan`, `jan`, `cdan`, `me_iis` (optional `pseudo_label`)

Each run writes artifacts to:
`outputs/runs/{dataset}/{src}2{tgt}/{method}/{run_id}/`

Every run saves `signature.json` (method routing guard), `metrics.csv`, logs, and checkpoints.


In [ ]:
import os, sys, subprocess
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

# Optional: mount Drive (set MOUNT_DRIVE=1) and point ME_IIS_DATA_ROOT to a Drive path.
if IN_COLAB and os.getenv("MOUNT_DRIVE", "0") == "1":
    try:
        from google.colab import drive  # type: ignore

        drive.mount("/content/drive")
    except Exception as exc:
        print("[Drive][WARN]", exc)

REPO_URL = os.getenv(
    "ME_IIS_REPO_URL",
    "https://github.com/kaminglui/Domain-Adaptation-with-ME-IIS.git",
)
REPO_DIR = Path(
    os.getenv(
        "ME_IIS_REPO_DIR",
        "/content/Domain-Adaptation-with-ME-IIS" if IN_COLAB else str(Path.cwd()),
    )
).expanduser()

if IN_COLAB:
    if not (REPO_DIR / ".git").exists():
        subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(REPO_DIR)], check=True)
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "-r", str(REPO_DIR / "requirements.txt")], check=True)
    os.chdir(REPO_DIR)

REPO_ROOT = Path.cwd().resolve()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("Repo root:", REPO_ROOT)
try:
    sha = subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=REPO_ROOT).decode().strip()
    print("git sha:", sha)
except Exception:
    pass


In [ ]:
import os, sys
from pathlib import Path

# ---------------- Single config cell ----------------
DATASET = os.getenv("ME_IIS_DATASET", "office_home")  # office_home | office31
DATA_ROOT = os.getenv("ME_IIS_DATA_ROOT", "")         # set this if not using repo-local datasets/
MODE = os.getenv("ME_IIS_MODE", "quick")              # quick | full

INCLUDE_PSEUDO = os.getenv("ME_IIS_INCLUDE_PSEUDO", "0") == "1"
METHODS = ["source_only", "dann", "dan", "jan", "cdan", "me_iis"]
if INCLUDE_PSEUDO:
    METHODS.append("pseudo_label")

SEEDS = [0] if MODE == "quick" else [0, 1, 2]

name = DATASET.lower().replace("-", "").replace("_", "")
if name == "officehome":
    DOMAINS = ["Ar", "Cl", "Pr", "Rw"]
elif name == "office31":
    DOMAINS = ["A", "D", "W"]
else:
    raise ValueError(f"Unknown dataset '{DATASET}'")

if MODE == "quick":
    DOMAIN_PAIRS = [("Ar", "Cl")] if DOMAINS == ["Ar", "Cl", "Pr", "Rw"] else [("A", "W")]
else:
    DOMAIN_PAIRS = [(s, t) for s in DOMAINS for t in DOMAINS if s != t]

FORCE_RERUN = os.getenv("ME_IIS_FORCE_RERUN", "0") == "1"
DEVICE = os.getenv("ME_IIS_DEVICE", "cuda")  # cuda | cpu | cuda:0
AMP = None  # None=default, True/False override

FREEZE_BACKBONE = os.getenv("ME_IIS_FREEZE_BACKBONE", "0") == "1"
BOTTLENECK_DIM = int(os.getenv("ME_IIS_BOTTLENECK_DIM", "256"))

EPOCHS_SOURCE = 1 if MODE == "quick" else 50
EPOCHS_ADAPT = 1 if MODE == "quick" else 10
BATCH_SIZE = 8 if MODE == "quick" else 32
NUM_WORKERS = 0 if MODE == "quick" else 4

DRY_RUN_MAX_SAMPLES = 256 if MODE == "quick" else 0
DRY_RUN_MAX_BATCHES = 0  # set >0 to cap optimizer steps for debugging

# Colab-first defaults: enable auto resource tuning and set device for all runs.
os.environ["ME_IIS_DEVICE"] = str(DEVICE)
if os.environ.get("ME_IIS_AUTO_RESOURCES") is None:
    os.environ["ME_IIS_AUTO_RESOURCES"] = "1" if ("google.colab" in sys.modules) else "0"

# Resolve dataset root and optionally cache Drive -> local SSD on Colab.
if not DATA_ROOT:
    if name == "officehome":
        DATA_ROOT = str(Path("datasets") / "Office-Home")
    elif name == "office31":
        DATA_ROOT = str(Path("datasets") / "Office-31")

from utils.resource_auto import maybe_cache_dataset_to_local

resolved_root, cache_info = maybe_cache_dataset_to_local(dataset_name=DATASET, data_root=Path(DATA_ROOT))
DATA_ROOT = str(resolved_root)

print("DATASET:", DATASET)
print("DATA_ROOT:", DATA_ROOT)
print("MODE:", MODE, "| seeds:", SEEDS, "| methods:", METHODS, "| pairs:", len(DOMAIN_PAIRS))
print("CACHE:", cache_info)


In [ ]:
import json
from pathlib import Path

from utils.resource_auto import detect_resources
from src.experiments.run_config import default_runs_root

runs_root = default_runs_root()
snap = detect_resources(disk_path=runs_root, data_path=Path(DATA_ROOT), cuda_device=0)
print(json.dumps(snap.to_json_dict(), indent=2, sort_keys=True))
print("runs_root:", runs_root)


In [ ]:
import time
from src.experiments.run_config import RunConfig, get_run_dir
from src.experiments.runner import run_one

# Override to place runs elsewhere (default is Colab-local SSD when available).
RUNS_ROOT = None


def make_cfg(method: str, src: str, tgt: str, seed: int) -> RunConfig:
    method_params = {}
    if AMP is not None:
        method_params["amp"] = bool(AMP)
    if method == "me_iis":
        method_params.update(
            {
                "feature_layers": ["layer3", "layer4"],
                "iis_iters": 15 if MODE != "quick" else 5,
                "gmm_selection_mode": "fixed",
                "num_latent_styles": 5,
                "cluster_clean_ratio": 1.0,
            }
        )
    if method == "pseudo_label":
        method_params.update(
            {
                "pseudo_conf_thresh": 0.9,
                "pseudo_max_ratio": 1.0,
                "pseudo_loss_weight": 1.0,
            }
        )
    epochs_adapt = 0 if method == "source_only" else int(EPOCHS_ADAPT)
    return RunConfig(
        dataset_name=str(DATASET),
        data_root=str(DATA_ROOT),
        source_domain=str(src),
        target_domain=str(tgt),
        method=str(method),
        epochs_source=int(EPOCHS_SOURCE),
        epochs_adapt=int(epochs_adapt),
        batch_size=int(BATCH_SIZE),
        num_workers=int(NUM_WORKERS),
        bottleneck_dim=int(BOTTLENECK_DIM),
        finetune_backbone=bool(not FREEZE_BACKBONE),
        method_params=method_params,
        seed=int(seed),
        deterministic=True,
        dry_run_max_samples=int(DRY_RUN_MAX_SAMPLES),
        dry_run_max_batches=int(DRY_RUN_MAX_BATCHES),
    )


configs = []
results = []

for (src, tgt) in DOMAIN_PAIRS:
    for seed in SEEDS:
        for method in METHODS:
            cfg = make_cfg(method, src, tgt, seed)
            configs.append(cfg)
            run_dir = get_run_dir(cfg, runs_root=RUNS_ROOT)
            print(f"[RUN] {method} {src}->{tgt} seed={seed} run_id={cfg.run_id} dir={run_dir}")
            t0 = time.time()
            res = run_one(
                cfg,
                force_rerun=bool(FORCE_RERUN),
                runs_root=RUNS_ROOT,
                write_metrics=True,
                raise_on_error=False,
            )
            res = dict(res)
            res["elapsed_sec"] = round(time.time() - t0, 3)
            results.append(res)
            status = res.get("status")
            extra = res.get("error") or ""
            print(f"  status={status} elapsed={res['elapsed_sec']}s {extra}")

print("Total runs:", len(results))


In [ ]:
from collections import defaultdict

from src.experiments.notebook_summary import collect_expected_runs

rows = collect_expected_runs(configs, runs_root=RUNS_ROOT)

try:
    import pandas as pd  # type: ignore

    df = pd.DataFrame(rows)
    display(df["status"].value_counts())
    display(df[["dataset", "src", "tgt", "method", "seed", "status", "target_acc", "run_dir"]])

    ok = df[df["status"] == "OK"].copy()
    ok["target_acc"] = ok["target_acc"].astype(float)

    # Per-task table (index: transfer+seed, columns: method)
    pivot = ok.pivot_table(index=["src", "tgt", "seed"], columns="method", values="target_acc")
    display(pivot)

    # Paper-style: 12-transfer average per seed, then mean±std across seeds.
    per_seed = ok.groupby(["method", "seed"])["target_acc"].mean().reset_index()
    summary = per_seed.groupby("method")["target_acc"].agg(["mean", "std", "count"]).reset_index()
    summary = summary.rename(columns={"mean": "mean_12transfer", "std": "std_across_seeds", "count": "num_seeds"})
    display(summary.sort_values("mean_12transfer", ascending=False))

    bad = df[df.get("invalid_comparison", False) == True]
    if len(bad):
        display(bad[["dataset", "src", "tgt", "seed", "method", "signature_fingerprint", "invalid_with_methods"]])
except Exception as exc:
    print("[WARN] pandas summary unavailable:", exc)
    ok = [r for r in rows if r.get("status") == "OK" and str(r.get("target_acc", "")).strip()]
    by_method_seed = defaultdict(list)
    for r in ok:
        by_method_seed[(r["method"], int(r["seed"]))].append(float(r["target_acc"]))

    import statistics

    out = []
    for method in METHODS:
        seed_means = []
        for seed in SEEDS:
            vals = by_method_seed.get((method, seed), [])
            if vals:
                seed_means.append(sum(vals) / len(vals))
        if seed_means:
            out.append(
                {
                    "method": method,
                    "mean_12transfer": sum(seed_means) / len(seed_means),
                    "std_across_seeds": statistics.pstdev(seed_means) if len(seed_means) > 1 else 0.0,
                    "num_seeds": len(seed_means),
                }
            )
    for row in sorted(out, key=lambda x: x["mean_12transfer"], reverse=True):
        print(row)


In [ ]:
import json
from pathlib import Path

from src.experiments.run_config import get_run_dir

samples = {}
for cfg in configs:
    run_dir = Path(get_run_dir(cfg, runs_root=RUNS_ROOT))
    sig_path = run_dir / "signature.json"
    if not sig_path.exists():
        continue
    sig = json.loads(sig_path.read_text(encoding="utf-8"))
    method = sig.get("method_name", cfg.method)
    if method in samples:
        continue
    samples[method] = {
        "amp": sig.get("method_params_used", {}).get("amp"),
        "dataloader": sig.get("dataloader"),
        "step_budget": sig.get("step_budget"),
        "comparison_fingerprint": sig.get("comparison_fingerprint"),
    }

samples
